In [2]:
import torch
import torch.nn.functional as F
from torchvision import transforms  # 是一个常用的图片变换类
from torchvision import datasets
from torch.utils.data import DataLoader
import prettytable

Side_length=8
my_batch_size = Side_length*Side_length

MyTransform = transforms.Compose(
    [
        transforms.ToTensor(),  # 把数据转换成张量
        transforms.Normalize((0.1307,), (0.3081,))  # 0.1307是均值，0.3081是标准差，具体如何计算得到，之后再看
    ]
)
train_dataset = datasets.MNIST(root='./dataset/mnist',train=True,
            download=False, transform=MyTransform)  # #第一次使用需要设置为true，下载数据集,使用的图片变换类transforms
train_process_data = DataLoader(train_dataset,  
            shuffle=True, batch_size=my_batch_size)   #shuffle：是否将数据打乱   #batch_size设置为my_batch_size
test_dataset = datasets.MNIST(root='./dataset/mnist',train=False,
            download=False,transform=MyTransform)
test_process_data = DataLoader(test_dataset,
            shuffle=True,batch_size=my_batch_size)

class MyCNN_NET(torch.nn.Module):
    def __init__(self):
        super(MyCNN_NET, self).__init__()  #初始化函数为空

        self.L1_conv_pool = torch.nn.Sequential(  # torch.nn.Sequential理解为向网络中增加结构
            torch.nn.Conv2d(1, 26, kernel_size=3), #卷积层，输入通道数目，输出通道数目，卷积核大小为3*3
            torch.nn.BatchNorm2d(26),   #输入batch的每一个特征通道进行normalize，对25个通道进行normalize标准化
            torch.nn.ReLU(inplace=True),  #激活函数relu  inplace为True，将会改变输入的数据 

            torch.nn.MaxPool2d(kernel_size=2, stride=2)  #kernel_size ：表示做最大池化的窗口大小，可以是单个值，也可以是tuple元组
                                                            #stride ：步长，可以是单个值，也可以是tuple元组
        )
        self.L2_conv_pool = torch.nn.Sequential(
            torch.nn.Conv2d(26, 52, kernel_size=3),  #输出通道数50
            torch.nn.BatchNorm2d(52),
            torch.nn.ReLU(inplace=True) ,
            torch.nn.MaxPool2d(kernel_size=2, stride=2) 
        )
        self.L3_fc = torch.nn.Sequential(  #全连接层
            torch.nn.Linear(52 * 5 * 5, 1024),  #由1250*1024变到1024*128，变到128*10
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )
    
    def ForwardPropagation(self, x):
        x = self.L1_conv_pool(x)
        x = self.L2_conv_pool(x)
        x = x.view(x.size(0), -1)  # 在进入全连接层之前需要把数据拉直Flatten   view()的作用相当于numpy中的reshape，重新定义矩阵的形状。
        #变行数为x.size(0)，列数随着其自动调整
        x = self.L3_fc(x)
        return x

In [6]:

def myTest():
    print("test===0")
    correct_sum = 0
    total_sum = 0
    with torch.no_grad():  # torch.no_grad()是一个上下文管理器，用来禁止梯度的计算，通常用来网络推断中，它可以减少计算内存的使用量。
        for data in test_process_data:
            Feature, Tag = data
            # Feature, Tag = Feature.to(device), Tag.to(device)  # 将数据放在GPU上跑所需要的代码
            outputs = model_CNN2.ForwardPropagation(Feature)
            Row_Max , Max_Index = torch.max(outputs.data, dim=1)  # Row_Max为每一行的最大值，predicted表示每一行最大值的下标
            #input是softmax函数输出的一个tensor ， dim是max函数索引的维度0/1，0是每列的最大值，1是每行的最大值
            total_sum += Tag.size(0)  #计算样本总数
            correct_sum += (Max_Index == Tag).sum().item()   
            mypredict.append(Max_Index)                 
            realtarget.append(Tag)               
    print(type(correct_sum))
    print(type(total_sum))
    print('Accuracy :%.6f %%' % (100 * correct_sum / total_sum))

In [8]:
if __name__ == '__main__':
    realtarget=[]
    mypredict=[]
    model_CNN2 = MyCNN_NET()
    model_weight_path = "./modelpth/model_new_18.pth"
    model_CNN2 = torch.load(model_weight_path)
    myTest()
    

test===0
<class 'int'>
<class 'int'>
Accuracy :99.400000 %
